Some thoughts:

* Is attention all you need? That may be true for seq2seq learning, but not in text summarization

* Since other Kagglers found adding attention to LSTM helps, why not add the multi head attention to LSTM?

* Blending rocks (multi head attention)


Reference:

* Attention Is All You Need: https://arxiv.org/abs/1706.03762

* Transformer in Keras: https://github.com/Lsdefine/attention-is-all-you-need-keras/blob/master/transformer.py
    
* SRK: https://www.kaggle.com/sudalairajkumar/a-look-at-different-embeddings . There is not much changed from this kernel except the nueral net architecture and final weights of the embeddings.

* Attention model from Khoi Ngyuen: https://www.kaggle.com/suicaokhoailang/lstm-attention-baseline-0-652-lb

* https://www.kaggle.com/shujian/different-embeddings-with-attention-fork-fork

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
# print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [2]:
import os
import time
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
import math
from sklearn.model_selection import train_test_split
from sklearn import metrics

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, CuDNNLSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D, Flatten, GlobalAveragePooling1D, Reshape
from keras.optimizers import Adam
from keras.models import Model
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints, optimizers, layers

Using TensorFlow backend.


In [3]:
# train_df = pd.read_csv("train.csv")
# val_df = pd.read_csv("valid.csv")
# test_df = pd.read_csv("test.csv")
# print("Train shape : ",train_df.shape)
# print("Valid shape : ",val_df.shape)
# print("Test shape : ",test_df.shape)

s_df = pd.read_csv("allsides.tsv",sep='\t')
df = pd.read_csv("debate_text.tsv",sep='\t')

print("raw data shape: ", df.shape)

raw data shape:  (34740, 50)


In [4]:
# Let's start with debt
target_df = df[:10000].dropna()
target = "Abortion"

target_labels = target_df[target].values
index_to_remove = np.where(target_labels > 1)
target_df = target_df.drop(index_to_remove[0])


## split to train and val
train_df, val_df = train_test_split(target_df, test_size=0.1, random_state=2019)
pretrain_df = s_df.dropna()


## some config values 
embed_size = 300 # how big is each word vector
max_features = 68000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 1500 # max number of words in a question to use


## fill up the missing values
pretrain_X = pretrain_df["text"].fillna("_##_").values
train_X = train_df["text"].fillna("_##_").values
val_X = val_df["text"].fillna("_##_").values


## Tokenize the sentences
tokenizer = Tokenizer(num_words=max_features)
fit_text = list(train_X)
fit_text.extend(list(pretrain_X))
tokenizer.fit_on_texts(fit_text)

pretrain_X = tokenizer.texts_to_sequences(pretrain_X)
train_X = tokenizer.texts_to_sequences(train_X)
val_X = tokenizer.texts_to_sequences(val_X)


## Pad the sentences 
pretrain_X = pad_sequences(pretrain_X, maxlen=maxlen)
train_X = pad_sequences(train_X, maxlen=maxlen)
val_X = pad_sequences(val_X, maxlen=maxlen)


import keras

## Get the target values
# train_y = train_df['stars'].apply(int)-1
# val_y = val_df['stars'].apply(int)-1
#test_y = test_df['stars'].apply(int)-1

train_y = train_df[target].values
pretrain_y = pretrain_df["label"].values
    
val_y = val_df[target].values

# train_y = keras.utils.to_categorical(train_y, num_classes=3)
# val_y = keras.utils.to_categorical(val_y, num_classes=3)
#test_y = keras.utils.to_categorical(test_y, num_classes=5)


In [5]:
print(len(train_X))
print(len(val_X))
print(len(train_y))
print(len(val_y))

2793
311
2793
311


In [6]:
#shuffling the data
np.random.seed(2019)
trn_idx = np.random.permutation(len(train_X))
val_idx = np.random.permutation(len(val_X))

train_X = train_X[trn_idx]
val_X = val_X[val_idx]
train_y = train_y[trn_idx]
val_y = val_y[val_idx]


In [7]:
EMBEDDING_FILE = 'glove.840B.300d.txt'
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE, encoding='utf-8'))

all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= (max_features): continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

C:\Users\xzhao\Anaconda3\envs\Keras\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  """


In [8]:
import random, os, sys
import numpy as np
from keras.models import *
from keras.layers import *
from keras.callbacks import *
from keras.initializers import *
import tensorflow as tf
from keras.engine.topology import Layer

try:
    from dataloader import TokenList, pad_to_longest
    # for transformer
except: pass

class LayerNormalization(Layer):
    def __init__(self, eps=1e-6, **kwargs):
        self.eps = eps
        super(LayerNormalization, self).__init__(**kwargs)
    def build(self, input_shape):
        self.gamma = self.add_weight(name='gamma', shape=input_shape[-1:],
                                     initializer=Ones(), trainable=True)
        self.beta = self.add_weight(name='beta', shape=input_shape[-1:],
                                    initializer=Zeros(), trainable=True)
        super(LayerNormalization, self).build(input_shape)
    def call(self, x):
        mean = K.mean(x, axis=-1, keepdims=True)
        std = K.std(x, axis=-1, keepdims=True)
        return self.gamma * (x - mean) / (std + self.eps) + self.beta
    def compute_output_shape(self, input_shape):
        return input_shape

class ScaledDotProductAttention():
    def __init__(self, d_model, attn_dropout=0.1):
        self.temper = np.sqrt(d_model)
        self.dropout = Dropout(attn_dropout)
    def __call__(self, q, k, v, mask):
        attn = Lambda(lambda x:K.batch_dot(x[0],x[1],axes=[2,2])/self.temper)([q, k])
        if mask is not None:
            mmask = Lambda(lambda x:(-1e+10)*(1-x))(mask)
            attn = Add()([attn, mmask])
        attn = Activation('softmax')(attn)
        attn = self.dropout(attn)
        output = Lambda(lambda x:K.batch_dot(x[0], x[1]))([attn, v])
        return output, attn

class MultiHeadAttention():
    # mode 0 - big martixes, faster; mode 1 - more clear implementation
    def __init__(self, n_head, d_model, d_k, d_v, dropout, mode=0, use_norm=True):
        self.mode = mode
        self.n_head = n_head
        self.d_k = d_k
        self.d_v = d_v
        self.dropout = dropout
        if mode == 0:
            self.qs_layer = Dense(n_head*d_k, use_bias=False)
            self.ks_layer = Dense(n_head*d_k, use_bias=False)
            self.vs_layer = Dense(n_head*d_v, use_bias=False)
        elif mode == 1:
            self.qs_layers = []
            self.ks_layers = []
            self.vs_layers = []
            for _ in range(n_head):
                self.qs_layers.append(TimeDistributed(Dense(d_k, use_bias=False)))
                self.ks_layers.append(TimeDistributed(Dense(d_k, use_bias=False)))
                self.vs_layers.append(TimeDistributed(Dense(d_v, use_bias=False)))
        self.attention = ScaledDotProductAttention(d_model)
        self.layer_norm = LayerNormalization() if use_norm else None
        self.w_o = TimeDistributed(Dense(d_model))

    def __call__(self, q, k, v, mask=None):
        d_k, d_v = self.d_k, self.d_v
        n_head = self.n_head

        if self.mode == 0:
            qs = self.qs_layer(q)  # [batch_size, len_q, n_head*d_k]
            ks = self.ks_layer(k)
            vs = self.vs_layer(v)

            def reshape1(x):
                s = tf.shape(x)   # [batch_size, len_q, n_head * d_k]
                x = tf.reshape(x, [s[0], s[1], n_head, d_k])
                x = tf.transpose(x, [2, 0, 1, 3])  
                x = tf.reshape(x, [-1, s[1], d_k])  # [n_head * batch_size, len_q, d_k]
                return x
            qs = Lambda(reshape1)(qs)
            ks = Lambda(reshape1)(ks)
            vs = Lambda(reshape1)(vs)

            if mask is not None:
                mask = Lambda(lambda x:K.repeat_elements(x, n_head, 0))(mask)
            head, attn = self.attention(qs, ks, vs, mask=mask)  
                
            def reshape2(x):
                s = tf.shape(x)   # [n_head * batch_size, len_v, d_v]
                x = tf.reshape(x, [n_head, -1, s[1], s[2]]) 
                x = tf.transpose(x, [1, 2, 0, 3])
                x = tf.reshape(x, [-1, s[1], n_head*d_v])  # [batch_size, len_v, n_head * d_v]
                return x
            head = Lambda(reshape2)(head)
        elif self.mode == 1:
            heads = []; attns = []
            for i in range(n_head):
                qs = self.qs_layers[i](q)   
                ks = self.ks_layers[i](k) 
                vs = self.vs_layers[i](v) 
                head, attn = self.attention(qs, ks, vs, mask)
                heads.append(head); attns.append(attn)
            head = Concatenate()(heads) if n_head > 1 else heads[0]
            attn = Concatenate()(attns) if n_head > 1 else attns[0]

        outputs = self.w_o(head)
        outputs = Dropout(self.dropout)(outputs)
        if not self.layer_norm: return outputs, attn
        # outputs = Add()([outputs, q]) # sl: fix
        return self.layer_norm(outputs), attn

class PositionwiseFeedForward():
    def __init__(self, d_hid, d_inner_hid, dropout=0.1):
        self.w_1 = Conv1D(d_inner_hid, 1, activation='relu')
        self.w_2 = Conv1D(d_hid, 1)
        self.layer_norm = LayerNormalization()
        self.dropout = Dropout(dropout)
    def __call__(self, x):
        output = self.w_1(x) 
        output = self.w_2(output)
        output = self.dropout(output)
        output = Add()([output, x])
        return self.layer_norm(output)

class EncoderLayer():
#class Encoder():
    def __init__(self, d_model, d_inner_hid, n_head, d_k, d_v, dropout=0.1):
        self.self_att_layer = MultiHeadAttention(n_head, d_model, d_k, d_v, dropout=dropout)
        self.pos_ffn_layer  = PositionwiseFeedForward(d_model, d_inner_hid, dropout=dropout)
    def __call__(self, enc_input, mask=None):
        output, slf_attn = self.self_att_layer(enc_input, enc_input, enc_input, mask=mask)
        output = self.pos_ffn_layer(output)
        return output, slf_attn

class Encoder():
    def __init__(self, d_model, d_inner_hid, n_head, d_k, d_v, \
                layers=6, dropout=0.1, word_emb=None, pos_emb=None):
        self.emb_layer = word_emb
        self.pos_layer = pos_emb
        self.emb_dropout = Dropout(dropout)
        self.layers = [EncoderLayer(d_model, d_inner_hid, n_head, d_k, d_v, dropout) for _ in range(layers)]
    def __call__(self, src_seq, src_pos, return_att=False, active_layers=999):
        x = self.emb_layer(src_seq)
        if src_pos is not None:
            pos = self.pos_layer(src_pos)
            x = Add()([x, pos])
        x = self.emb_dropout(x)
        if return_att: atts = []
        mask = Lambda(lambda x:GetPadMask(x, x))(src_seq)
        for enc_layer in self.layers[:active_layers]:
            x, att = enc_layer(x, mask)
            if return_att: atts.append(att)
        return (x, atts) if return_att else x
    
    

def GetPosEncodingMatrix(max_len, d_emb):
    pos_enc = np.array([
        [pos / np.power(10000, 2 * (j // 2) / d_emb) for j in range(d_emb)] 
        if pos != 0 else np.zeros(d_emb) 
            for pos in range(max_len)
            ])
    pos_enc[1:, 0::2] = np.sin(pos_enc[1:, 0::2]) # dim 2i
    pos_enc[1:, 1::2] = np.cos(pos_enc[1:, 1::2]) # dim 2i+1
    return pos_enc

def GetPadMask(q, k):
    ones = K.expand_dims(K.ones_like(q, 'float32'), -1)
    mask = K.cast(K.expand_dims(K.not_equal(k, 0), 1), 'float32')
    mask = K.batch_dot(ones, mask, axes=[2,1])
    return mask

def GetSubMask(s):
    len_s = tf.shape(s)[1]
    bs = tf.shape(s)[:1]
    mask = K.cumsum(tf.eye(len_s, batch_shape=bs), 1)
    return mask

class Transformer():
    def __init__(self, len_limit, embedding_matrix, d_model=embed_size, \
              d_inner_hid=512, n_head=10, d_k=64, d_v=64, layers=2, dropout=0.1, \
              share_word_emb=False, **kwargs):
        self.name = 'Transformer'
        self.len_limit = len_limit
        self.src_loc_info = False # True # sl: fix later
        self.d_model = d_model
        self.decode_model = None
        d_emb = d_model

        pos_emb = Embedding(len_limit, d_emb, trainable=False, \
                            weights=[GetPosEncodingMatrix(len_limit, d_emb)])

        i_word_emb = Embedding(max_features, d_emb, weights=[embedding_matrix]) # Add Kaggle provided embedding here
        # i_word_emb = Embedding(68976, d_emb, weights=[embedding_matrix])
        
        self.encoder = Encoder(d_model, d_inner_hid, n_head, d_k, d_v, layers, dropout, \
                               word_emb=i_word_emb, pos_emb=pos_emb)

        
    def get_pos_seq(self, x):
        mask = K.cast(K.not_equal(x, 0), 'int32')
        pos = K.cumsum(K.ones_like(x, 'int32'), 1)
        return pos * mask

    def compile(self, active_layers=999):
        src_seq_input = Input(shape=(None, ))
        x = Embedding(max_features, embed_size, weights=[embedding_matrix])(src_seq_input)
        # x = Embedding(65359, embed_size, weights=[embedding_matrix])(src_seq_input)
        
        # LSTM before attention layers
        # x = Bidirectional(CuDNNLSTM(128, return_sequences=True))(x)
        x = Bidirectional(CuDNNLSTM(300, return_sequences=True))(x)
        x = Bidirectional(CuDNNLSTM(64, return_sequences=True))(x) 
        
        x, slf_attn = MultiHeadAttention(n_head=3, d_model=300, d_k=64, d_v=64, dropout=0.1)(x, x, x)
        
        avg_pool = GlobalAveragePooling1D()(x)
        max_pool = GlobalMaxPooling1D()(x)
        conc = concatenate([avg_pool, max_pool])
        conc = Dense(64, activation="relu")(conc)
        x = Dense(1, activation="sigmoid")(conc)   
        
        self.model = Model(inputs=src_seq_input, outputs=x)
        # self.model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        self.model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [9]:
s2s = Transformer(64, embedding_matrix, layers=1)
s2s.compile()
model = s2s.model
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, None, 300)    20400000    input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, None, 600)    1444800     embedding_3[0][0]                
__________________________________________________________________________________________________
bidirectional_2 (Bidi

In [10]:
## Train the model 
model.fit(pretrain_X, pretrain_y, batch_size=20, epochs=4,verbose=1,validation_data=(val_X, val_y))
model.fit(train_X, train_y, batch_size=20, epochs=4,verbose=1,validation_data=(val_X, val_y))


Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 4879 samples, validate on 311 samples
Epoch 1/4


4140/4879 [========================>.....] - ETA: 20:39 - loss: 0.8047 - acc: 0.50 - ETA: 11:27 - loss: 0.8446 - acc: 0.55 - ETA: 8:23 - loss: 0.8642 - acc: 0.5167 - ETA: 6:51 - loss: 0.9041 - acc: 0.512 - ETA: 5:55 - loss: 0.8801 - acc: 0.500 - ETA: 5:17 - loss: 0.8341 - acc: 0.533 - ETA: 4:50 - loss: 0.8662 - acc: 0.521 - ETA: 4:30 - loss: 0.8467 - acc: 0.537 - ETA: 4:14 - loss: 0.8230 - acc: 0.550 - ETA: 4:01 - loss: 0.8240 - acc: 0.555 - ETA: 3:51 - loss: 0.8615 - acc: 0.536 - ETA: 3:42 - loss: 0.8651 - acc: 0.525 - ETA: 3:34 - loss: 0.8471 - acc: 0.530 - ETA: 3:28 - loss: 0.8367 - acc: 0.532 - ETA: 3:22 - loss: 0.8289 - acc: 0.530 - ETA: 3:17 - loss: 0.8163 - acc: 0.540 - ETA: 3:12 - loss: 0.8118 - acc: 0.526 - ETA: 3:08 - loss: 0.8067 - acc: 0.519 - ETA: 3:04 - loss: 0.7987 - acc: 0.526 - ETA: 3:01 - loss: 0.7974 - acc: 0.520 - ETA: 2:58 - loss: 0.7924 - acc: 0.521 - ETA: 2:55 - loss: 0.7873 - acc: 0.520 - ETA: 2:52 - loss: 0.7808 - acc: 0.523 - ETA: 2:49 - loss: 0.7767 - acc: 0.

4879/4879 [==============================] - ETA: 21s - loss: 0.6938 - acc: 0.57 - ETA: 21s - loss: 0.6935 - acc: 0.57 - ETA: 20s - loss: 0.6934 - acc: 0.57 - ETA: 20s - loss: 0.6931 - acc: 0.57 - ETA: 19s - loss: 0.6934 - acc: 0.57 - ETA: 18s - loss: 0.6935 - acc: 0.57 - ETA: 18s - loss: 0.6937 - acc: 0.57 - ETA: 17s - loss: 0.6932 - acc: 0.57 - ETA: 17s - loss: 0.6928 - acc: 0.57 - ETA: 16s - loss: 0.6926 - acc: 0.57 - ETA: 15s - loss: 0.6922 - acc: 0.57 - ETA: 15s - loss: 0.6924 - acc: 0.57 - ETA: 14s - loss: 0.6930 - acc: 0.57 - ETA: 14s - loss: 0.6930 - acc: 0.57 - ETA: 13s - loss: 0.6930 - acc: 0.57 - ETA: 12s - loss: 0.6928 - acc: 0.57 - ETA: 12s - loss: 0.6927 - acc: 0.57 - ETA: 11s - loss: 0.6922 - acc: 0.57 - ETA: 10s - loss: 0.6919 - acc: 0.57 - ETA: 10s - loss: 0.6917 - acc: 0.57 - ETA: 9s - loss: 0.6917 - acc: 0.5774 - ETA: 9s - loss: 0.6917 - acc: 0.577 - ETA: 8s - loss: 0.6916 - acc: 0.576 - ETA: 7s - loss: 0.6918 - acc: 0.575 - ETA: 7s - loss: 0.6918 - acc: 0.576 - ETA:

4140/4879 [========================>.....] - ETA: 2:23 - loss: 0.6610 - acc: 0.650 - ETA: 2:22 - loss: 0.6470 - acc: 0.675 - ETA: 2:22 - loss: 0.6721 - acc: 0.633 - ETA: 2:21 - loss: 0.6583 - acc: 0.650 - ETA: 2:21 - loss: 0.6419 - acc: 0.670 - ETA: 2:20 - loss: 0.6556 - acc: 0.641 - ETA: 2:20 - loss: 0.6533 - acc: 0.642 - ETA: 2:20 - loss: 0.6552 - acc: 0.637 - ETA: 2:19 - loss: 0.6551 - acc: 0.633 - ETA: 2:19 - loss: 0.6605 - acc: 0.625 - ETA: 2:18 - loss: 0.6611 - acc: 0.622 - ETA: 2:18 - loss: 0.6598 - acc: 0.625 - ETA: 2:18 - loss: 0.6597 - acc: 0.623 - ETA: 2:17 - loss: 0.6607 - acc: 0.621 - ETA: 2:16 - loss: 0.6626 - acc: 0.613 - ETA: 2:16 - loss: 0.6586 - acc: 0.621 - ETA: 2:15 - loss: 0.6614 - acc: 0.617 - ETA: 2:14 - loss: 0.6697 - acc: 0.600 - ETA: 2:14 - loss: 0.6702 - acc: 0.600 - ETA: 2:13 - loss: 0.6695 - acc: 0.597 - ETA: 2:12 - loss: 0.6708 - acc: 0.585 - ETA: 2:12 - loss: 0.6730 - acc: 0.579 - ETA: 2:11 - loss: 0.6718 - acc: 0.580 - ETA: 2:11 - loss: 0.6762 - acc: 0.5

4879/4879 [==============================] - ETA: 21s - loss: 0.6738 - acc: 0.57 - ETA: 20s - loss: 0.6737 - acc: 0.57 - ETA: 20s - loss: 0.6737 - acc: 0.57 - ETA: 19s - loss: 0.6735 - acc: 0.58 - ETA: 19s - loss: 0.6736 - acc: 0.58 - ETA: 18s - loss: 0.6734 - acc: 0.58 - ETA: 17s - loss: 0.6737 - acc: 0.57 - ETA: 17s - loss: 0.6734 - acc: 0.58 - ETA: 16s - loss: 0.6733 - acc: 0.58 - ETA: 16s - loss: 0.6736 - acc: 0.58 - ETA: 15s - loss: 0.6733 - acc: 0.58 - ETA: 14s - loss: 0.6737 - acc: 0.58 - ETA: 14s - loss: 0.6736 - acc: 0.58 - ETA: 13s - loss: 0.6734 - acc: 0.58 - ETA: 13s - loss: 0.6730 - acc: 0.58 - ETA: 12s - loss: 0.6729 - acc: 0.58 - ETA: 11s - loss: 0.6726 - acc: 0.58 - ETA: 11s - loss: 0.6726 - acc: 0.58 - ETA: 10s - loss: 0.6726 - acc: 0.58 - ETA: 10s - loss: 0.6728 - acc: 0.58 - ETA: 9s - loss: 0.6727 - acc: 0.5811 - ETA: 8s - loss: 0.6729 - acc: 0.581 - ETA: 8s - loss: 0.6732 - acc: 0.580 - ETA: 7s - loss: 0.6733 - acc: 0.580 - ETA: 7s - loss: 0.6738 - acc: 0.580 - ETA:

...

4140/4879 [========================>.....] - ETA: 2:26 - loss: 0.4912 - acc: 0.800 - ETA: 2:25 - loss: 0.4617 - acc: 0.825 - ETA: 2:24 - loss: 0.5022 - acc: 0.766 - ETA: 2:24 - loss: 0.5170 - acc: 0.750 - ETA: 2:24 - loss: 0.5441 - acc: 0.710 - ETA: 2:24 - loss: 0.5300 - acc: 0.725 - ETA: 2:23 - loss: 0.5247 - acc: 0.735 - ETA: 2:22 - loss: 0.5214 - acc: 0.743 - ETA: 2:21 - loss: 0.5063 - acc: 0.755 - ETA: 2:21 - loss: 0.5116 - acc: 0.745 - ETA: 2:20 - loss: 0.5080 - acc: 0.754 - ETA: 2:19 - loss: 0.4918 - acc: 0.770 - ETA: 2:19 - loss: 0.4822 - acc: 0.776 - ETA: 2:18 - loss: 0.4815 - acc: 0.785 - ETA: 2:18 - loss: 0.4722 - acc: 0.796 - ETA: 2:17 - loss: 0.4802 - acc: 0.790 - ETA: 2:17 - loss: 0.4705 - acc: 0.800 - ETA: 2:16 - loss: 0.4588 - acc: 0.805 - ETA: 2:15 - loss: 0.4660 - acc: 0.797 - ETA: 2:15 - loss: 0.4766 - acc: 0.790 - ETA: 2:14 - loss: 0.4736 - acc: 0.792 - ETA: 2:13 - loss: 0.4674 - acc: 0.793 - ETA: 2:13 - loss: 0.4684 - acc: 0.789 - ETA: 2:12 - loss: 0.4727 - acc: 0.7

4879/4879 [==============================] - ETA: 21s - loss: 0.4605 - acc: 0.78 - ETA: 20s - loss: 0.4610 - acc: 0.77 - ETA: 20s - loss: 0.4612 - acc: 0.77 - ETA: 19s - loss: 0.4610 - acc: 0.77 - ETA: 19s - loss: 0.4607 - acc: 0.77 - ETA: 18s - loss: 0.4607 - acc: 0.77 - ETA: 17s - loss: 0.4598 - acc: 0.77 - ETA: 17s - loss: 0.4597 - acc: 0.78 - ETA: 16s - loss: 0.4590 - acc: 0.78 - ETA: 16s - loss: 0.4586 - acc: 0.78 - ETA: 15s - loss: 0.4581 - acc: 0.78 - ETA: 14s - loss: 0.4572 - acc: 0.78 - ETA: 14s - loss: 0.4577 - acc: 0.78 - ETA: 13s - loss: 0.4584 - acc: 0.77 - ETA: 13s - loss: 0.4577 - acc: 0.78 - ETA: 12s - loss: 0.4579 - acc: 0.77 - ETA: 11s - loss: 0.4573 - acc: 0.78 - ETA: 11s - loss: 0.4565 - acc: 0.78 - ETA: 10s - loss: 0.4566 - acc: 0.78 - ETA: 10s - loss: 0.4571 - acc: 0.78 - ETA: 9s - loss: 0.4572 - acc: 0.7811 - ETA: 8s - loss: 0.4568 - acc: 0.781 - ETA: 8s - loss: 0.4568 - acc: 0.781 - ETA: 7s - loss: 0.4573 - acc: 0.781 - ETA: 7s - loss: 0.4578 - acc: 0.781 - ETA:

2793/2793 [==============================] - ETA: 1:22 - loss: 0.6845 - acc: 0.500 - ETA: 1:22 - loss: 0.6914 - acc: 0.525 - ETA: 1:21 - loss: 0.6918 - acc: 0.533 - ETA: 1:22 - loss: 0.6885 - acc: 0.537 - ETA: 1:21 - loss: 0.6991 - acc: 0.490 - ETA: 1:21 - loss: 0.6908 - acc: 0.516 - ETA: 1:20 - loss: 0.6872 - acc: 0.521 - ETA: 1:19 - loss: 0.6850 - acc: 0.531 - ETA: 1:19 - loss: 0.6840 - acc: 0.533 - ETA: 1:18 - loss: 0.6845 - acc: 0.535 - ETA: 1:17 - loss: 0.6869 - acc: 0.513 - ETA: 1:17 - loss: 0.6873 - acc: 0.516 - ETA: 1:16 - loss: 0.6872 - acc: 0.519 - ETA: 1:16 - loss: 0.6868 - acc: 0.521 - ETA: 1:15 - loss: 0.6871 - acc: 0.520 - ETA: 1:14 - loss: 0.6893 - acc: 0.512 - ETA: 1:14 - loss: 0.6861 - acc: 0.523 - ETA: 1:13 - loss: 0.6828 - acc: 0.530 - ETA: 1:12 - loss: 0.6834 - acc: 0.531 - ETA: 1:12 - loss: 0.6796 - acc: 0.540 - ETA: 1:11 - loss: 0.6814 - acc: 0.540 - ETA: 1:11 - loss: 0.6834 - acc: 0.540 - ETA: 1:10 - loss: 0.6886 - acc: 0.534 - ETA: 1:09 - loss: 0.6897 - acc: 0.5

2793/2793 [==============================] - ETA: 1:22 - loss: 0.6453 - acc: 0.600 - ETA: 1:21 - loss: 0.5926 - acc: 0.725 - ETA: 1:20 - loss: 0.6280 - acc: 0.650 - ETA: 1:21 - loss: 0.6242 - acc: 0.675 - ETA: 1:20 - loss: 0.6329 - acc: 0.660 - ETA: 1:20 - loss: 0.6419 - acc: 0.633 - ETA: 1:19 - loss: 0.6264 - acc: 0.657 - ETA: 1:18 - loss: 0.6235 - acc: 0.662 - ETA: 1:18 - loss: 0.6180 - acc: 0.677 - ETA: 1:18 - loss: 0.6218 - acc: 0.660 - ETA: 1:17 - loss: 0.6158 - acc: 0.668 - ETA: 1:17 - loss: 0.6194 - acc: 0.675 - ETA: 1:16 - loss: 0.6199 - acc: 0.673 - ETA: 1:15 - loss: 0.6092 - acc: 0.682 - ETA: 1:15 - loss: 0.6068 - acc: 0.683 - ETA: 1:14 - loss: 0.5995 - acc: 0.693 - ETA: 1:13 - loss: 0.6125 - acc: 0.676 - ETA: 1:13 - loss: 0.6024 - acc: 0.683 - ETA: 1:12 - loss: 0.5949 - acc: 0.689 - ETA: 1:12 - loss: 0.5993 - acc: 0.690 - ETA: 1:11 - loss: 0.6096 - acc: 0.683 - ETA: 1:10 - loss: 0.6271 - acc: 0.670 - ETA: 1:10 - loss: 0.6392 - acc: 0.660 - ETA: 1:09 - loss: 0.6445 - acc: 0.6

2793/2793 [==============================] - ETA: 1:23 - loss: 0.7178 - acc: 0.400 - ETA: 1:22 - loss: 0.7081 - acc: 0.450 - ETA: 1:21 - loss: 0.7001 - acc: 0.500 - ETA: 1:20 - loss: 0.7014 - acc: 0.487 - ETA: 1:20 - loss: 0.7036 - acc: 0.470 - ETA: 1:19 - loss: 0.6995 - acc: 0.491 - ETA: 1:18 - loss: 0.6975 - acc: 0.514 - ETA: 1:18 - loss: 0.6978 - acc: 0.500 - ETA: 1:17 - loss: 0.6965 - acc: 0.516 - ETA: 1:17 - loss: 0.6964 - acc: 0.515 - ETA: 1:16 - loss: 0.6955 - acc: 0.518 - ETA: 1:16 - loss: 0.6951 - acc: 0.525 - ETA: 1:15 - loss: 0.6942 - acc: 0.530 - ETA: 1:14 - loss: 0.6939 - acc: 0.528 - ETA: 1:14 - loss: 0.6930 - acc: 0.533 - ETA: 1:13 - loss: 0.6944 - acc: 0.525 - ETA: 1:12 - loss: 0.6939 - acc: 0.526 - ETA: 1:12 - loss: 0.6962 - acc: 0.511 - ETA: 1:11 - loss: 0.6959 - acc: 0.513 - ETA: 1:11 - loss: 0.6952 - acc: 0.515 - ETA: 1:11 - loss: 0.6950 - acc: 0.516 - ETA: 1:10 - loss: 0.6951 - acc: 0.511 - ETA: 1:09 - loss: 0.6951 - acc: 0.510 - ETA: 1:09 - loss: 0.6950 - acc: 0.5

In [11]:
# pred_val_y = model.predict([val_X], batch_size=20, verbose=1)

valid_score = model.evaluate(val_X, val_y, batch_size=20)

print(valid_score)
print('Validation Loss: {}\n Validation Accuracy: {}\n'.format(valid_score[0], valid_score[1]))

# thresholds = []
# for thresh in np.arange(0.1, 0.501, 0.01):
#     thresh = np.round(thresh, 2)
#     res = metrics.f1_score(val_y, (pred_val_y > thresh).astype(int))
#     thresholds.append([thresh, res])
#     print("F1 score at threshold {0} is {1}".format(thresh, res))
    
# thresholds.sort(key=lambda x: x[1], reverse=True)
# best_thresh = thresholds[0][0]
# print("Best threshold: ", best_thresh)

311/311 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 3s 11ms/step
[0.6891399209905668, 0.5659163967972783]
Validation Loss: 0.6891399209905668
 Validation Accuracy: 0.5659163967972783



In [12]:
print(valid_score)

[0.6891399209905668, 0.5659163967972783]


In [ ]:
pred_test_y = model.predict([test_X], batch_size=1024, verbose=1)
pred_test_y = pred_test_y.argmax(axis=-1) + 1
out_df = pd.DataFrame({"review_id":test_df["review_id"].values})
out_df['pred'] = pred_test_y
out_df.to_csv("pred.csv", index=False)